## Word2Vec

Набор данных содержит равное количество позитивных и негативных твитов, общее количество которых составляет ~200 000. Мы случайным образом отбираем позитивные и негативные твиты, чтобы разделить исходный набор данных на два набора для обучения и тестирования в соотношении 80/20. Затем обучаем модель Word2Vec на обучающем наборе данных.

Чтобы предотвратить утечку данных из тестового набора, мы не обучаем Word2Vec на тестовом наборе, пока наш классификатор не обучится на обучающем наборе. Чтобы создать входные данные для классификатора, используем усредненный вектор, полученный из всех векторов слов в твите. Для реализации машинного обучения будем использовать библиотеку scikit-learn.

Вначале импортируем данные и обучаем модель Word2Vec.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec

with open('extra_data/prepared_tweets.txt', 'r') as infile:
    tweets = infile.readlines()
    
with open('extra_data/targets.txt', 'r') as infile:
    y = infile.readlines()

/Users/maksim/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
x_train, x_test, y_train, y_test = train_test_split(tweets, y, test_size=0.2)

In [4]:
print "x_train num:", len(x_train)
print "y_train num:", len(y_train)
print "x_test num:", len(x_test)
print "x_test num:", len(y_test)

x_train num: 181467
y_train num: 181467
x_test num: 45367
x_test num: 45367


In [5]:
def cleanText(corpus):
    corpus = [z.lower().replace('\n','').split() for z in corpus]
    return corpus

x_train = cleanText(x_train)
x_test = cleanText(x_test)

In [6]:
y_train = [int(y.strip()) for y in y_train]
y_test = [int(y.strip()) for y in y_test]

In [8]:
from gensim.models.word2vec import Word2Vec

# список параметров, которые можно менять по вашему желанию
num_features = 512  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

w2v = Word2Vec(workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

In [9]:
#build vocab
w2v.build_vocab(x_train)

In [10]:
#Train the model over train_reviews (this may take several minutes)
w2v.train(x_train)

7917082

In [ ]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWordVector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

Масштабирование – это часть процесса стандартизации, в рамках которого мы приводим наш набор данных к гауссовому распределению со средним значением, равным нулю.

Как следствие, значения, превышающие среднее, будут положительными, а те, что меньше него, – отрицательными. Для эффективной работы многих моделей машинного обучения требуются масштабированные наборы данных, особенно для моделей с большим количеством признаков, таких как текстовые классификаторы.

In [ ]:
from sklearn.preprocessing import scale
train_vecs = np.concatenate([buildWordVector(z, num_features) for z in x_train])
train_vecs = scale(train_vecs)

#Train word2vec on test tweets
w2v.train(x_test)

Мы должны создать векторы тестового набора и масштабировать их для оценки.

In [ ]:
#Build test tweet vectors then scale
test_vecs = np.concatenate([buildWordVector(z, num_features) for z in x_test])
test_vecs = scale(test_vecs)

Далее мы проверим наш классификатор, рассчитав точность прогнозирования на тестовых данных, а также проанализируем ROC-кривую. ROC-кривая характеризует соотношение истинноположительных и ложноположительных результатов классификации при варьировании параметра модели.

В нашем случае мы регулируем пороговую вероятность классификации твита, как позитивного или негативного. В общем случае, чем больше площадь под кривой (AUC), тем лучше наша модель максимизирует количество истинноположительных и минимизирует количество ложноположительных результатов. 

Для начала мы обучаем наш классификатор, используя в данном случае стохастический градиентный спуск (stochastic gradient descent) для логистической регрессии.

In [ ]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vecs, y_train)

print 'Test Accuracy: %.2f'%lr.score(test_vecs, y_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
pipe = Pipeline([("algo", LogisticRegression())])

param_dict = {
              'algo__C': [0.01, 0.1, 1, 10, 100],
              'algo__penalty': ['l2', 'l1']}

estimator = GridSearchCV(pipe, param_dic

In [ ]:
#Create ROC curve
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

pred_probas = lr.predict_proba(test_vecs)[:,1]

fpr,tpr,_ = roc_curve(y_test, pred_probas)
roc_auc = auc(fpr,tpr)
plt.plot(fpr,tpr,label='area = %.2f' %roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')

plt.show()